# RAG Application with LangChain and HuggingFace *LLM*

In [ ]:
# Instaall the nessary pckage
!pip install torch -q
!pip install transformers -q
!pip install numpy -q
!pip install langchain -q
!pip install langchain_community -q
!pip install sentence_transformers -q
!pip install langchain-chroma -q



## Initialize Huggingface LLM

https://huggingface.co/mistralai/Mistral-7B-v0.1

In [ ]:
from langchain_community.llms import HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    model_kwargs={"temperature":0.1, "max_length":500},
    huggingfacehub_api_token="<API_TOKEN>"
)

## Initialize Embedding Model

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
)

## **Initialize Output Parser**

In [9]:
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

## Load PDF Documents

In [ ]:
!pip install pypdf -qU

In [82]:
from langchain_community.document_loaders import PyPDFLoader

pdf_files = ["/content/Networking.pdf", "/content/aws-overview.pdf"]
docs = []

for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    docs.extend(loader.load())

In [ ]:
docs

## Split Documents into Chunks

In [83]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 400,
    chunk_overlap  = 50,
)

splits = text_splitter.split_documents(docs)

## Create Vector Store and Retriever

In [84]:
from langchain_chroma import Chroma

# Create a vector store from the documnent chunks

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model,
)


In [85]:
# Create a retriever from the vector store

retriever = vectorstore.as_retriever()

## Define Prompt Template

In [86]:
from langchain.prompts import ChatPromptTemplate

# Define prompt template

template = """
Answer this question using the provide context only.

{question}

Context:
{context}

Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
prompt

## Chain Retriver and Prompt Template With LLM

In [88]:
from langchain.schema.runnable import RunnablePassthrough

# Create a chain

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)


## Invoke RAG Chain with Exmple Questions

In [ ]:
response = chain.invoke(input="What is Amazon Virtual Private Cloud (Amazon VPC), and why is it important in AWS networking?")
print(response)

In [ ]:
response = chain.invoke(input="How does a virtual private network (VPN) connection add security to a VPC?")
print(response)

In [ ]:
response = chain.invoke(input="What is AWS Fargate, and how does it simplify container management?")
print(response)

In [ ]:
response = chain.invoke(input="What are the two modes available in Amazon ECS for running containers?")
print(response)